## Data Preprocessing

In [1]:
import sys
sys.path.insert(0, '..')
import os

import numpy as np
import pandas as pd

In [2]:
import zipfile
with zipfile.ZipFile('./data/labeledTrainData.tsv.zip', 'r') as z:
    z.extractall('./data/')

In [3]:
data_train = pd.read_csv('./data/labeledTrainData.tsv', sep='\t')

In [4]:
print(data_train.shape)
data_train.head(5)

(25000, 3)


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [5]:
data_train['review'][0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [6]:
data_train.review.shape[0]

25000

In [7]:
from bs4 import BeautifulSoup
import re

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets 
    Every dataset is lower cased
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

texts = []
labels = []
for idx in range(data_train.review.shape[0]):
    text = BeautifulSoup(data_train.review[idx], "lxml")
    texts.append(clean_str(text.get_text()))
    labels.append(data_train.sentiment[idx])

In [8]:
len(texts)

25000

In [9]:
texts[0]

"with all this stuff going down at the moment with mj i 've started listening to his music , watching the odd documentary here and there , watched the wiz and watched moonwalker again maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent moonwalker is part biography , part feature film which i remember going to see at the cinema when it was originally released some of it has subtle messages about mj 's feeling towards the press and also the obvious message of drugs are bad m'kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for 20 m

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Dropout, Concatenate
from keras.layers import Bidirectional, LSTM, GRU, CuDNNLSTM, CuDNNGRU
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

C:\Users\sun\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

C:\Users\sun\Anaconda3\lib\site-packages\keras_preprocessing\text.py:175: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 81988 unique tokens.


In [12]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (25000, 1000)
Shape of label tensor: (25000, 2)


In [13]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set ')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Number of positive and negative reviews in traing and validation set 
[ 9981. 10019.]
[2519. 2481.]


In [14]:
GLOVE_DIR = "../pre_trained word embeddings/glove.6B"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

Total 400000 word vectors in Glove 6B 100d.


In [15]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [16]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

## Deep Model

In [21]:
from keras.models import Sequential
from keras.layers import Convolution1D, Dropout, Activation 

NB_FILTER = 256
FILTER_LENGTH = 5

print ("Training model.")
model = Sequential()
model.add(embedding_layer)
model.add(Convolution1D(nb_filter=NB_FILTER,
                        filter_length=FILTER_LENGTH,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
# use max pooling:
model.add(MaxPooling1D(pool_length=model.output_shape[1]))
# We flatten the output of the conv layer,
# so that we can add a vanilla dense layer:
model.add(Flatten())
# We add a vanilla hidden layer:
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(2, activation='sigmoid'))

model.summary() 

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.fit(x_train, y_train, validation_data=(x_val, y_val),nb_epoch=10, batch_size=128)

Training model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 100)         8198900   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 996, 256)          128256    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 1, 256)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0    

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=256, kernel_size=5, strides=1, padding="valid")`
  from ipykernel import kernelapp as app
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=996)`
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 23s 1ms/step - loss: 0.3280 - acc: 0.8358 - val_loss: 0.3656 - val_acc: 0.8817
Epoch 2/10
20000/20000 [==============================] - 22s 1ms/step - loss: 0.0751 - acc: 0.9739 - val_loss: 0.4081 - val_acc: 0.8796
Epoch 3/10
20000/20000 [==============================] - 22s 1ms/step - loss: 0.0251 - acc: 0.9913 - val_loss: 0.6895 - val_acc: 0.8639
Epoch 4/10
20000/20000 [==============================] - 22s 1ms/step - loss: 0.0097 - acc: 0.9970 - val_loss: 0.7501 - val_acc: 0.8730
Epoch 5/10
20000/20000 [==============================] - 22s 1ms/step - loss: 0.0130 - acc: 0.9955 - val_loss: 0.7973 - val_acc: 0.8491
Epoch 6/10
20000/20000 [==============================] - 22s 1ms/step - loss: 0.0088 - acc: 0.9970 - val_loss: 0.8852 - val_acc: 0.8798
Epoch 7/10
20000/20000 [==============================] - 22s 1ms/step - loss: 0.0039 - acc: 0.9988 - val_loss: 0.8906 - val_acc:

In [31]:
# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(pool_length=MAX_SEQUENCE_LENGTH-fsz+1)(l_conv)
    convs.append(l_pool)
l_merge = Concatenate(axis=1)(convs) 
l_flat = Flatten()(l_merge)
l_dense1= Dense(256, activation='relu')(l_flat)
l_dropout1 = Dropout(0.5)(l_dense1)
l_dense = Dense(128, activation='relu')(l_dropout1)
l_dropout2 = Dropout(0.3)(l_dense)
preds = Dense(2, activation='softmax')(l_dropout2)
model = Model(sequence_input, preds)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.summary() 

model.fit(x_train, y_train, validation_data=(x_val, y_val),nb_epoch=10, batch_size=128)

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=3)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=998)`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=4)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=997)`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="rel

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 100)    8198900     input_8[0][0]                    
__________________________________________________________________________________________________
conv1d_25 (Conv1D)              (None, 998, 128)     38528       embedding_1[11][0]               
__________________________________________________________________________________________________
conv1d_26 (Conv1D)              (None, 997, 128)     51328       embedding_1[11][0]               
__________________________________________________________________________________________________
conv1d_27 

In [24]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dropout1 = Dropout(0.5)(l_flat)
l_dense = Dense(128, activation='relu')(l_dropout1)
l_dropout2 = Dropout(0.3)(l_dense)
preds = Dense(2, activation='softmax')(l_dropout2)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
print("model fitting - simplified convolutional neural network")
model.summary()

model fitting - simplified convolutional neural network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8198900   
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_9 (Conv1D)   

In [25]:
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=5, batch_size=128)

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 11s 554us/step - loss: 0.4344 - acc: 0.7590 - val_loss: 0.3536 - val_acc: 0.8756
Epoch 2/5
20000/20000 [==============================] - 10s 483us/step - loss: 0.1241 - acc: 0.9559 - val_loss: 0.4720 - val_acc: 0.8718
Epoch 3/5
20000/20000 [==============================] - 10s 482us/step - loss: 0.0504 - acc: 0.9832 - val_loss: 0.6448 - val_acc: 0.8688
Epoch 4/5
20000/20000 [==============================] - 10s 485us/step - loss: 0.0323 - acc: 0.9896 - val_loss: 0.7590 - val_acc: 0.8674
Epoch 5/5
20000/20000 [==============================] - 10s 485us/step - loss: 0.0161 - acc: 0.9945 - val_loss: 0.8486 - val_acc: 0.8684


In [26]:
# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)
l_merge = Concatenate(axis=1)(convs)    
# l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(30)(l_cov2)
l_flat = Flatten()(l_pool2)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(2, activation='softmax')(l_dense)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - more complex convolutional neural network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=3)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=4)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=5)`
  if __name__ == '__main__':
C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


model fitting - more complex convolutional neural network
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 100)    8198900     input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 998, 128)     38528       embedding_1[6][0]                
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 997, 128)     51328       embedding_1[6][0]                
___________________________________________________

## Bi-LSTM

In [32]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm = Bidirectional(CuDNNLSTM(100))(embedded_sequences)
preds = Dense(2, activation='softmax')(l_lstm)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - Bidirectional LSTM")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - Bidirectional LSTM
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8198900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               161600    
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 402       
Total params: 8,360,902
Trainable params: 8,360,902
Non-trainable params: 0
_________________________________________________________________


C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  del sys.path[0]


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 50s 2ms/step - loss: 0.3962 - acc: 0.8282 - val_loss: 0.3171 - val_acc: 0.8844
Epoch 2/10
20000/20000 [==============================] - 49s 2ms/step - loss: 0.1533 - acc: 0.9480 - val_loss: 0.3891 - val_acc: 0.8710
Epoch 3/10
20000/20000 [==============================] - 49s 2ms/step - loss: 0.0722 - acc: 0.9763 - val_loss: 0.3632 - val_acc: 0.8862
Epoch 4/10
20000/20000 [==============================] - 49s 2ms/step - loss: 0.0394 - acc: 0.9868 - val_loss: 0.5082 - val_acc: 0.8852
Epoch 5/10
20000/20000 [==============================] - 48s 2ms/step - loss: 0.0278 - acc: 0.9913 - val_loss: 0.5018 - val_acc: 0.8818
Epoch 6/10
20000/20000 [==============================] - 49s 2ms/step - loss: 0.0162 - acc: 0.9950 - val_loss: 0.5982 - val_acc: 0.8870
Epoch 7/10
20000/20000 [==============================] - 48s 2ms/step - loss: 0.0110 - acc: 0.9965 - val_loss: 0.6488 - val_acc:

## Attention

In [17]:
# use Theano as backend
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        eij = K.tanh(K.dot(x, self.W))
        
        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [18]:
class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [18]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_gru = Bidirectional(CuDNNGRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_gru)
preds = Dense(2, activation='softmax')(l_att)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention GRU network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - attention GRU network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8198900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1000, 200)         121200    
_________________________________________________________________
att_layer_1 (AttLayer)       (None, 200)               20200     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 8,340,702
Trainable params: 8,340,702
Non-trainable params: 0
_________________________________________________________________


C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 56s 3ms/step - loss: 0.4143 - acc: 0.7980 - val_loss: 0.3316 - val_acc: 0.8580
Epoch 2/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.2492 - acc: 0.8982 - val_loss: 0.2708 - val_acc: 0.8908
Epoch 3/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.1908 - acc: 0.9245 - val_loss: 0.2374 - val_acc: 0.9082
Epoch 4/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.1473 - acc: 0.9433 - val_loss: 0.2282 - val_acc: 0.9110
Epoch 5/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.1056 - acc: 0.9618 - val_loss: 0.2360 - val_acc: 0.9172
Epoch 6/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0717 - acc: 0.9756 - val_loss: 0.3170 - val_acc: 0.9088
Epoch 7/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0429 - acc: 0.9867 - val_loss: 0.3978 - val_acc:

In [19]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_gru = Bidirectional(CuDNNGRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_gru)
l_dropout = Dropout(0.2)(l_att)
preds = Dense(2, activation='softmax')(l_dropout)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention GRU network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - attention GRU network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8198900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1000, 200)         121200    
_________________________________________________________________
att_layer_1 (AttLayer)       (None, 200)               20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 8,340,702
Trainable params: 8,340,702
Non-trainable params: 0
____________________________

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 56s 3ms/step - loss: 0.4028 - acc: 0.8090 - val_loss: 0.3140 - val_acc: 0.8760
Epoch 2/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.2531 - acc: 0.8980 - val_loss: 0.2896 - val_acc: 0.8766
Epoch 3/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.1931 - acc: 0.9250 - val_loss: 0.2176 - val_acc: 0.9130
Epoch 4/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.1511 - acc: 0.9445 - val_loss: 0.2365 - val_acc: 0.9096
Epoch 5/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.1135 - acc: 0.9600 - val_loss: 0.2374 - val_acc: 0.9120
Epoch 6/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0811 - acc: 0.9723 - val_loss: 0.2763 - val_acc: 0.9056
Epoch 7/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0525 - acc: 0.9831 - val_loss: 0.3241 - val_acc:

In [19]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_gru = Bidirectional(CuDNNGRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_gru)
l_dropout = Dropout(0.5)(l_att)
preds = Dense(2, activation='softmax')(l_dropout)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention GRU network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - attention GRU network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8198900   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1000, 200)         121200    
_________________________________________________________________
att_layer_2 (AttLayer)       (None, 200)               20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
Total params: 8,340,702
Trainable params: 8,340,702
Non-trainable params: 0
____________________________

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.2513 - acc: 0.8897 - val_loss: 0.2434 - val_acc: 0.9054
Epoch 2/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0943 - acc: 0.9660 - val_loss: 0.2964 - val_acc: 0.9054
Epoch 3/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0518 - acc: 0.9816 - val_loss: 0.3474 - val_acc: 0.9014
Epoch 4/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0294 - acc: 0.9901 - val_loss: 0.4211 - val_acc: 0.8990
Epoch 5/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0165 - acc: 0.9954 - val_loss: 0.5425 - val_acc: 0.8940
Epoch 6/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0096 - acc: 0.9976 - val_loss: 0.6227 - val_acc: 0.9008
Epoch 7/10
20000/20000 [==============================] - 54s 3ms/step - loss: 0.0061 - acc: 0.9979 - val_loss: 0.7327 - val_acc:

In [20]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_gru = Bidirectional(CuDNNGRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_gru)
l_dropout = Dropout(0.2)(l_att)
preds = Dense(2, activation='softmax')(l_dropout)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("model fitting - attention GRU network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - attention GRU network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8198900   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1000, 200)         121200    
_________________________________________________________________
att_layer_2 (AttLayer)       (None, 200)               20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
Total params: 8,340,702
Trainable params: 8,340,702
Non-trainable params: 0
____________________________

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 56s 3ms/step - loss: 0.2345 - acc: 0.8959 - val_loss: 0.2277 - val_acc: 0.9122
Epoch 2/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0684 - acc: 0.9763 - val_loss: 0.3465 - val_acc: 0.9014
Epoch 3/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0218 - acc: 0.9934 - val_loss: 0.3927 - val_acc: 0.9014
Epoch 4/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0095 - acc: 0.9972 - val_loss: 0.4575 - val_acc: 0.8980
Epoch 5/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0050 - acc: 0.9985 - val_loss: 0.5368 - val_acc: 0.9026
Epoch 6/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0032 - acc: 0.9990 - val_loss: 0.6356 - val_acc: 0.9026
Epoch 7/10
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0033 - acc: 0.9988 - val_loss: 0.6061 - val_acc:

In [21]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_gru = Bidirectional(CuDNNGRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_gru)
l_dropout = Dropout(0.5)(l_att)
preds = Dense(2, activation='softmax')(l_dropout)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("model fitting - attention GRU network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=5, batch_size=50)

model fitting - attention GRU network
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         8198900   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 1000, 200)         121200    
_________________________________________________________________
att_layer_3 (AttLayer)       (None, 200)               20200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 402       
Total params: 8,340,702
Trainable params: 8,340,702
Non-trainable params: 0
____________________________

C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 56s 3ms/step - loss: 0.1802 - acc: 0.9134 - val_loss: 0.3161 - val_acc: 0.9062
Epoch 2/5
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0184 - acc: 0.9939 - val_loss: 0.3991 - val_acc: 0.9048
Epoch 3/5
20000/20000 [==============================] - 55s 3ms/step - loss: 0.0058 - acc: 0.9983 - val_loss: 0.5415 - val_acc: 0.8988
Epoch 4/5
20000/20000 [==============================] - 55s 3ms/step - loss: 9.3853e-04 - acc: 0.9998 - val_loss: 0.6161 - val_acc: 0.9032
Epoch 5/5
20000/20000 [==============================] - 55s 3ms/step - loss: 1.4262e-04 - acc: 1.0000 - val_loss: 0.6886 - val_acc: 0.9034


## HATT

In [36]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from nltk import tokenize

MAX_SENT_LENGTH = 100
MAX_SENTS = 15

reviews = []
labels = []
texts = []

for idx in range(data_train.review.shape[0]):
    text = BeautifulSoup(data_train.review[idx], "lxml")
    text = clean_str(text.get_text())
    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    reviews.append(sentences)
    labels.append(data_train.sentiment[idx])

data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
                    data[i, j, k] = tokenizer.word_index[word]
                    k = k + 1

word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Total 81988 unique tokens.
Shape of data tensor: (25000, 15, 100)
Shape of label tensor: (25000, 2)
Number of positive and negative reviews in traing and validation set
[10003.  9997.]
[2497. 2503.]


In [37]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=True,
                            mask_zero=True)

In [38]:
class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [29]:
from keras.layers import TimeDistributed

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_lstm)
sentEncoder = Model(sentence_input, l_att)

review_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
l_att_sent = AttLayer(100)(l_lstm_sent)
preds = Dense(2, activation='softmax')(l_att_sent)
model = Model(review_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - Hierachical attention network")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

ValueError: Dimensions must be equal, but are 15 and 100 for 'att_layer_8/mul' (op: 'Mul') with input shapes: [?,15], [?,15,100].

In [35]:
from keras.layers import TimeDistributed

sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_dense = TimeDistributed(Dense(200))(l_lstm)
l_att = AttLayer(10)(l_dense)
sentEncoder = Model(sentence_input, l_att)

review_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
l_dense_sent = TimeDistributed(Dense(200))(l_lstm_sent)
l_att_sent = AttLayer(10)(l_dense_sent)
preds = Dense(2, activation='softmax')(l_att_sent)
model = Model(review_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.summary()

print("model fitting - Hierachical attention network")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 15, 100)           0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, 15, 200)           8313020   
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 15, 200)           180600    
_________________________________________________________________
time_distributed_16 (TimeDis (None, 15, 200)           40200     
_________________________________________________________________
att_layer_16 (AttLayer)      (None, 200)               2020      
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 402       
Total params: 8,536,242
Trainable params: 8,536,242
Non-trainable params: 0
_________________________________________________________________


C:\Users\sun\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 20000 samples, validate on 5000 samples
Epoch 1/10


InvalidArgumentError: Inputs to operation bidirectional_19/while/Select_1 of type Select must have the same size and shape.  Input 0: [50,10000] != input 1: [50,100]
	 [[Node: bidirectional_19/while/Select_1 = Select[T=DT_FLOAT, _class=["loc:@training_4/RMSprop/gradients/AddN_7"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](bidirectional_19/while/Tile, bidirectional_19/while/add_6, bidirectional_19/while/Switch_3:1)]]
	 [[Node: loss_4/mul/_1385 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_6143_loss_4/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]